In [1]:
import re
from datetime import datetime

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)
from pandas.io.json import json_normalize
import json
import ast

import plotly.graph_objects as go
import chart_studio.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

C:\Users\Eya\AppData\Local\Temp/ipykernel_14256/1403358917.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
commits = pd.read_csv('data/commits.csv', parse_dates=True)

C:\Users\Eya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (22,23) have mixed types.Specify dtype option on import or set low_memory=False.



In [3]:
commits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31336 entries, 0 to 31335
Data columns (total 62 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     31336 non-null  int64  
 1   sha                            31336 non-null  object 
 2   node_id                        31336 non-null  object 
 3   url                            31336 non-null  object 
 4   html_url                       31336 non-null  object 
 5   comments_url                   31336 non-null  object 
 6   parents                        31336 non-null  object 
 7   repo_name                      31336 non-null  object 
 8   owner                          31336 non-null  object 
 9   commit.author.name             31336 non-null  object 
 10  commit.author.email            31335 non-null  object 
 11  commit.author.date             31336 non-null  object 
 12  commit.committer.name          31336 non-null 

## Data Preprocessing

In [4]:
commits['date'] =  pd.to_datetime(commits['commit.committer.date'])

In [5]:
commits['date'] =  pd.to_datetime(commits['date'], utc=True)

In [6]:
commits['commit_date'] = commits['date'].dt.date

In [7]:
commits['commit_week'] = commits['date'].dt.week

C:\Users\Eya\AppData\Local\Temp/ipykernel_14256/575324427.py:1: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



In [8]:
commits['commit_hour'] = commits['date'].dt.hour

In [9]:
commits['commit_month'] = commits['date'].dt.month

In [10]:
commits['commit_year'] = commits['date'].dt.year

In [11]:
# drop unnecessary columns
commits = commits[['sha', 'author.login', 'commit_date', 'commit_hour', 'commit_month', 'commit_year']]

In [12]:
commits.head()

,sha,author.login,commit_date,commit_hour,commit_month,commit_year
0,38d39812c176e4b52a08397f7936f87ea32930e7,ueshin,2021-10-03,3,10,2021
1,14d4ceeb73c44dd957246f36eea9ece527d8d8d7,sunchao,2021-10-02,6,10,2021
2,25db6b45c7636a1c62b6fd6ad189836b019374a3,fishmandev,2021-10-01,16,10,2021
3,73747ecb970595d49c478b0eb65f5132c8b0bf02,venkata91,2021-10-01,7,10,2021
4,13ddc916683c01a0c076c79e3c0e491395eec9ff,itholic,2021-10-01,5,10,2021


## Data Analysis

In [13]:
commits['author.login'].unique().size

1681

In [14]:
commits_by_hour = commits.groupby('commit_hour')[['sha']].count()
commits_by_hour = commits_by_hour.rename(columns = {'sha': 'commit_count'})

In [15]:
fig = go.Figure(
    go.Bar(x=commits_by_hour.index,
            y=commits_by_hour.commit_count,
            text= commits_by_hour.commit_count,
            textposition='auto')
)
fig.update_layout(
    title="Commits per hour",
    xaxis_title="hour",
    yaxis_title="commits count",
    xaxis_tickmode = 'linear'       
)
fig.show()

In [16]:
commits_by_day = commits.groupby('commit_date')[['sha']].count()
commits_by_day = commits_by_day.rename(columns = {'sha': 'commit_count'})

In [17]:
fig = go.Figure(
    go.Scatter(x=commits_by_day.index,
            y=commits_by_day.commit_count,
            text= commits_by_day.commit_count,
            fill='tozeroy')
)
fig.update_layout(
    title="Commits per Date",
    xaxis_title="Date",
    yaxis_title="commits count",     
)
fig.show()

In [18]:
yearly_commits = commits.groupby('commit_year')[['sha']].count()
yearly_commits = yearly_commits.rename(columns = {'sha': 'commits_count'})
yearly_commits['daily_avg'] = yearly_commits['commits_count']/365

In [19]:
fig = go.Figure(
    go.Scatter(x=yearly_commits.index,
            y=yearly_commits.daily_avg,
            text= yearly_commits.daily_avg)
)
fig.update_layout(
    title="Average Daily Commits by year",
    xaxis_title="Year",
    yaxis_title="Average number of Commits",
    xaxis_tickmode = 'linear')
fig.show()

In [20]:
commits_by_author = commits.groupby('author.login')[['sha']].count()
commits_by_author = commits_by_author.rename(columns = {'sha': 'commit_count'})
commits_by_author = commits_by_author.sort_values(by='commit_count', ascending=False)
top_authors = commits_by_author.head(30)

In [21]:
fig = go.Figure(
    go.Bar(x=top_authors.index,
           y=top_authors.commit_count)
)
fig.update_layout(
    title="Top Committers",
    xaxis_title="author",
    yaxis_title="commits count",
    xaxis_tickmode = 'linear',
    xaxis_tickangle=-40)
fig.show()

## Open Pull Requests

In [22]:
pulls = pd.read_csv('data/pulls.csv', parse_dates=True)

In [23]:
pulls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Columns: 296 entries, Unnamed: 0 to head.repo
dtypes: bool(16), float64(24), int64(16), object(240)
memory usage: 465.0+ KB


In [24]:
pulls['date'] = pd.to_datetime(pulls['created_at'])
pulls['date'] = pd.to_datetime(pulls['date'], utc=True)
pulls['pull_date'] = pulls['date'].dt.date

In [25]:
pulls_by_date = pulls.groupby('pull_date')[['id']].count()
pulls_by_date = pulls_by_date.rename(columns = {'id': 'commit_count'})

In [26]:
fig = go.Figure([go.Scatter(
    x=pulls_by_date.index, 
    y=pulls_by_date.commit_count, 
    text=pulls_by_date.commit_count)])
fig.update_layout(
    title = 'Open Pull Requests by Date', 
    xaxis_title = 'Date', 
    yaxis_title = 'Pulls Count')
fig.show()

### Pull Request Labels

**NOTES**:  
- ast.literal_eval converts string to list.
- The following two lines convert a Pandas column that contains list of dictionaries into a Pandas Dataframe where each dictionary corresponds to a dataframe row.

In [27]:
labels_list = [ast.literal_eval(i) for i in pulls['labels'].tolist() if i != '[]']

In [28]:
labels = [j for i in labels_list for j in i]

In [29]:
labels_df = pd.DataFrame(labels)

In [30]:
labels_by_name = labels_df.groupby('name')[['id']].count()
labels_by_name = labels_by_name.rename(columns = {'id': 'label_count'})
labels_by_name = labels_by_name.sort_values(by=['label_count'], ascending=False)

In [31]:
fig = go.Figure([go.Bar(
    x=labels_by_name.index, 
    y=labels_by_name.label_count)])
fig.update_layout(
    title = 'Pull Requests by Label', 
    xaxis_title = 'Labels', 
    yaxis_title = 'PR Count', 
    xaxis_tickmode = 'linear',
    xaxis_tickangle=-40)
fig.show()